# Example sparql queries for Netpune database

Structured Query Language (SPARQL) is a powerful query language used for querying, updating, and interacting with RDF (Resource Description Framework) data. RDF is a standard model for representing data on the web, and it's particularly well-suited for expressing relationships between resources. SPARQL allows you to retrieve and manipulate RDF data, making it an essential tool for working with semantic web data and graph databases.

The purpose of this notebook is to provide some simple templates for exploratory querying of the database, they are by no means exhaustive. 

## Configuration of the notebook

Before beginning it is important to check that this notebook can access the neptune database that holds the data. The cell below contains two diagnostic commands that if executed, should help determine this. If you cannot access the neptune database it is probably because the notebook is unable to access the VPC that the Neptune database is hosted in, or the IAM permissions for the notebook are not configured. 

#### Debug commands

In [ ]:
%status

In [ ]:
%graph_notebook_config 

### Basic Sparql queries for probing data

#### List all subjects & predicates

The following query lists all unique subjects and predicates. If the "LIMIT" is removed - this list may be comprehensive, but difficult to navigate.

In [ ]:
%%sparql

SELECT DISTINCT ?subject ?predicate
WHERE {
  ?subject ?predicate ?object
} 


#### Simple count

In [ ]:
%%sparql

SELECT (COUNT(*) AS ?count)
WHERE {
  ?subject ?predicate ?object
}


#### Retrieve specific types of a resource 

In [ ]:
%%sparql

SELECT ?subject ?predicate
WHERE {
  ?subject ?predicate "agent.48K" .
}


#### Retrieve queries that share a resource between them

In [ ]:
%%sparql 

SELECT *
WHERE {
  ?start ?predicate1 ?intermediate .
  ?intermediate ?predicate2 ?end .
}
LIMIT 1000


#### Retrieve the subject node with the most connections

In [ ]:
%%sparql

SELECT ?subject (COUNT(*) as ?count)
WHERE {
  ?subject ?predicate ?object .
}
GROUP BY ?subject
ORDER BY DESC(?count)
LIMIT 1


#### Visualising the subject node and it's connections

In [ ]:
%%sparql

SELECT ?subject ?predicate ?object
WHERE {
  <http://cat.nationalarchives.gov.uk/COAL.2022.N2CL.P.1> ?predicate ?object .
  BIND (<http://cat.nationalarchives.gov.uk/COAL.2022.N2CL.P.1> AS ?subject)
}



#### Finding the object node with the most connections 

In [ ]:
%%sparql


SELECT ?object (COUNT(?predicate) AS ?connections)
WHERE {
  ?subject ?predicate ?object .
}
GROUP BY ?object
ORDER BY DESC(?connections)
LIMIT 1



#### Visualising the node with the most connections 

In [ ]:
%%sparql

SELECT ?subject ?predicate ?object
WHERE {
  ?subject ?predicate ?object .
  FILTER (?object = <http://cat.nationalarchives.gov.uk/agent.S7>)
} LIMIT 1000


#### Finding objects with different subjects 

In [ ]:
%%sparql

SELECT ?s1 ?p1 ?o
WHERE {
  ?s1 ?p1 ?o .
  ?s2 ?p2 ?o .
  FILTER (?s1 != ?s2)
} LIMIT 1000


#### Query all triples  

In [ ]:
%%sparql

SELECT *
WHERE {
  ?subject ?predicate ?object
} LIMIT 10000 

#### Storing query results for further analysis 

In [ ]:
%%sparql --store-to sparql_results

SELECT DISTINCT ?subject ?predicate
WHERE {
  ?subject ?predicate ?object
} 


#### Using stored values for further analysis 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

bindings = sparql_results['results']['bindings']

predicates = [binding['predicate']['value'] for binding in bindings]

df = pd.DataFrame({'predicate': predicates})

predicate_counts = df['predicate'].value_counts()

%matplotlib inline

# adjust these to format graph
bar_width = 0.5  # Width of each bar
spacing = 1   # Space between bars
positions = range(len(predicate_counts))
bar_positions = [pos * (bar_width + spacing) for pos in positions]

plt.bar(bar_positions, predicate_counts.values, width=bar_width)

# Set the positions and labels for the ticks on the x-axis
plt.xticks(bar_positions, predicate_counts.index, rotation=45, ha="right")

plt.xlabel('Predicates')
plt.ylabel('Number of Instances')
plt.title('Number of Instances for Each Predicate')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

labels = [result['subject']['value'] for result in sparql_results['results']['bindings']]
predicates = [result['predicate']['value'] for result in sparql_results['results']['bindings']]

predicate_counts = {}
for predicate in predicates:
    if predicate in predicate_counts:
        predicate_counts[predicate] += 1
    else:
        predicate_counts[predicate] = 1

sorted_predicates = sorted(predicate_counts.items(), key=lambda x: x[1], reverse=True)

N = 10  # You can adjust this value to display the top N predicates
top_predicates = [predicate[0] for predicate in sorted_predicates[:N]]
top_counts = [predicate[1] for predicate in sorted_predicates[:N]]

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(top_counts, labels=top_predicates, autopct='%1.1f%%', startangle=140)

plt.title('Top Predicates in SPARQL Query Results', y=1.10)# Adjust the y value to control title position

plt.axis('equal')  # Equal aspect ratio ensures that the pie is drawn as a circle
plt.show()


#### What's wrong with an old fashioned table

In [ ]:
import pandas as pd

# Extract 'predicate' values
predicate_values = [result['predicate']['value'] for result in sparql_results['results']['bindings']]
df = pd.DataFrame({'Predicate': predicate_values})
predicate_counts = df['Predicate'].value_counts().reset_index()
predicate_counts.columns = ['Predicate', 'Count']
predicate_counts['Percentage'] = (predicate_counts['Count'] / predicate_counts['Count'].sum() * 100).round(2)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
# Display the DataFrame
print(predicate_counts)

#### An interesting query to show aspects of the "Coal 80" subset 

In [ ]:
%%sparql

PREFIX dct: <http://purl.org/dc/terms/>
PREFIX cat: <http://cat.nationalarchives.gov.uk/>
PREFIX ver: <http://purl.org/linked-data/version#>

describe ?personConcept ?personDescription ?name
where { ?personConcept dct:type cat:person-concept ;
       ver:currentVersion ?personDescription . } LIMIT 100

#### The big finale

In [ ]:
%%sparql --store-to all_results

SELECT *
WHERE {
  ?subject ?predicate ?object
}